# Machine Vision Fundamentals and Applications - test


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Display a sample image
plt.imshow(X_train[0])
plt.title('Sample Image from CIFAR-10')
plt.axis('off')
plt.show()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

print('Data loaded and preprocessed successfully.')

## Implementing Key Machine Vision Techniques


In [ ]:
# Install OpenCV if not already installed (run once)
# !pip install opencv-python

import cv2

# Take a sample image from the dataset (denormalize for OpenCV)
sample_img = (X_train[0] * 255).astype(np.uint8)

# Convert to grayscale
gray = cv2.cvtColor(sample_img, cv2.COLOR_RGB2GRAY)

# Apply Gaussian blur filter
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Edge detection with Canny
edges = cv2.Canny(blurred, 50, 150)

# Display results
fig, axs = plt.subplots(1, 3, figsize=(10, 4))
axs[0].imshow(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')
axs[0].axis('off')
axs[1].imshow(blurred, cmap='gray')
axs[1].set_title('Blurred')
axs[1].axis('off')
axs[2].imshow(edges, cmap='gray')
axs[2].set_title('Edges')
axs[2].axis('off')
plt.show()

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load pre-trained VGG16 model (without top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers to prevent retraining
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using augmented data
history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=10,
                    validation_data=(X_test, y_test))

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

print('Model training completed.')